# ПостНаука

In [71]:
%reload_ext autoreload
%autoreload 2

In [72]:
%matplotlib inline
import matplotlib.pyplot as plt

In [73]:
import artm
import hierarchy_utils
import os
import glob
import sys
import numpy as np
import sklearn

In [74]:
%%time

data_path = '../datasets/postnauka/postnauka.txt'
data_path_batches = 'pn_batches/'
if len(glob.glob(data_path_batches + "/*.batch")) < 1:
    batch_vectorizer = artm.BatchVectorizer(data_format='vowpal_wabbit', \
                data_path = data_path, batch_size = 225, target_folder=data_path_batches)
else:
    batch_vectorizer = artm.BatchVectorizer(data_path=data_path_batches, data_format='batches')

CPU times: user 461 ms, sys: 0 ns, total: 461 ms
Wall time: 459 ms


In [75]:
class_ids={'text':1.0, 'flat_tag':300.0, 'author': 5.0}

In [76]:
# quality measures of models
scores_list = []
"""
scores_list.append(artm.SparsityPhiScore(name='SparsityScoreText',
                                         class_id='ngramm',
                                         #topic_names=topics_subj
                                        ))
scores_list.append(artm.SparsityThetaScore(name='SparsityScoreThetaSubj',
                                           #topic_names=topics_subj
                                          ))

scores_list.append(artm.SparsityPhiScore(name='SparsityScoreAuthor', class_id='author'))
scores_list.append(artm.SparsityPhiScore(name='SparsityScoreTag', class_id='flat_tag'))
scores_list.append(artm.TopicKernelScore(name='TopicKernelText', class_id='text', probability_mass_threshold=0.25))
"""

scores_list.append(artm.TopTokensScore(name='TopTokensScoreText', class_id='text', num_tokens=15))
scores_list.append(artm.TopTokensScore(name='TopTokensTag', class_id='flat_tag', num_tokens=10))
scores_list.append(artm.PerplexityScore(name='PerplexityScore', class_ids=['text']))
scores_list.append(artm.PerplexityScore(name='PerplexityScore_Tag', class_ids=['flat_tag']))

In [77]:
dict_path = 'dictionary.txt'
if not os.path.isfile(dict_path):
    dictionary = artm.Dictionary('dictionary')
    dictionary.gather(batch_vectorizer.data_path)
    dictionary.save_text(dict_path)
else:
    dictionary = artm.Dictionary('dictionary')
    dictionary.load_text(dict_path)

### Тут строится 0 уровень

In [78]:
norm_topic_names=['topic_'+str(i) for i in range(0,19)]
background_names=['background_0']
topic_names0=norm_topic_names+background_names


In [79]:
topic_names0[-1:]

['background_0']

In [80]:
regularizers_list = []


#сглаживание фоновой темы по словам
regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTagRegBackground',class_ids=['text'],
                                                         topic_names=background_names,tau=1000)) 
#декоррелирование остальных тем по словам
regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiReg',class_ids=['text'],
                                                         topic_names=norm_topic_names,
                                                         tau=500000))


"""regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiTagReg',class_ids=['flat_tag'],tau=500000))
regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTagReg',class_ids=['flat_tag'],
                                                         topic_names=topic_names0[:-1],tau=-15))
regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTextReg',
                                                         class_ids=['text'],
                                                         #topic_names=topics_subj,
                                                         tau=0.01))

#regularizers_list.append(artm.SmoothSparseThetaRegularizer(name='SmoothThetaRegSubj',
                                                           #topic_names=topics_subj,
                                                           tau=0.01))"""

"regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiTagReg',class_ids=['flat_tag'],tau=500000))\nregularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTagReg',class_ids=['flat_tag'],\n                                                         topic_names=topic_names0[:-1],tau=-15))\nregularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTextReg',\n                                                         class_ids=['text'],\n                                                         #topic_names=topics_subj,\n                                                         tau=0.01))\n\n#regularizers_list.append(artm.SmoothSparseThetaRegularizer(name='SmoothThetaRegSubj',\n                                                           #topic_names=topics_subj,\n                                                           tau=0.01))"

In [81]:
%%time

topicNum0 = len(topic_names0)

hier = hierarchy_utils.hARTM(class_ids=class_ids,
             regularizers=regularizers_list,
             scores=scores_list, cache_theta=True,
             seed=37)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 32.2 µs


In [82]:
level0 = artm.ARTM(topic_names=topic_names0, class_ids=class_ids,
             regularizers=regularizers_list,
             scores=scores_list, cache_theta=True,
             seed=37)

In [83]:
%%time

level0.initialize(dictionary=dictionary)
level0.fit_offline(batch_vectorizer, num_collection_passes=1)

CPU times: user 5.19 s, sys: 0 ns, total: 5.19 s
Wall time: 2.33 s


In [84]:
%%time

level0.fit_offline(batch_vectorizer, num_collection_passes=10)

CPU times: user 48.1 s, sys: 0 ns, total: 48.1 s
Wall time: 19.2 s


In [85]:
theta = level0.get_phi(class_ids=['author'])
theta.shape

(0, 20)

In [86]:
output = ""
tokens = level0.score_tracker["TopTokensTag"].last_tokens
for topic_name in level0.topic_names:
    output += topic_name + ': '
    for word in tokens[topic_name]:    
        output += word + ' '
    output += '\n'
print(output)

topic_0: россия история история_россии философия политическая_философия география античность история_философии аристотель археология 
topic_1: эволюция биология антропология человек палеонтология антропогенез физиология иммунитет происхождение_человека эволюционная_биология 
topic_2: экономика сша религиоведение политика финансовый_кризис финансовый_рынок теория_игр экономическое_развитие демография новая_институциональная_экономика 
topic_3: культура литература массовая_культура культурология философия филология литературоведение кино искусство кинематограф 
topic_4: история христианство ссср политика религия германия власть франция востоковедение европа 
topic_5: астрономия астрофизика звезды космос вселенная галактика земля черные_дыры солнце солнечная_система 
topic_6: язык лингвистика филология русский_язык право юриспруденция урбанистика город римское_право грамматика 
topic_7: социология общество город социология_повседневности дюркгейм_эмиль вебер_макс гоффман_эрвинг социология

In [87]:
output = ""
tokens = level0.score_tracker['TopTokensScoreText'].last_tokens
for topic_name in level0.topic_names:
    output += topic_name + ': '
    for word in tokens[topic_name]:    
        output += word + ' '
    output += '\n'
print(output)

topic_0: россия русский политический философия российский государство война территория земля власть остров понятие место философ находить 
topic_1: вид эволюция группа организм раса эволюционный ген возникать находить отбор жить часть животное антропология период 
topic_2: экономический экономика страна компания цена рынок деньги рост экономист кризис сша решение банк развитие уровень 
topic_3: культура фильм искусство литература кино культурный музей русский смысл автор текст советский массовый герой писать 
topic_4: война политический власть исторический советский революция государство страна историк сталин народ германия имя политика начало 
topic_5: звезда галактика планета земля дыра черный масса солнце объект космический газ телескоп нейтронный вещество солнечный 
topic_6: русский право город словарь речь текст значение языковой часть лингвист глагол предложение форма вид место 
topic_7: социальный социология пространство отношение объект общество город событие социологический де

In [88]:
print(level0.score_tracker['PerplexityScore'].value[-1])
print(level0.score_tracker['PerplexityScore_Tag'].value[-1])

3817.5408594133005
60.58319519002361


In [89]:
level0.theta_columns_naming='title'

In [90]:
level0.get_theta()

,pn_2701,pn_2702,pn_2703,pn_2704,pn_2705,pn_2706,pn_2707,pn_2708,pn_2709,pn_2710,...,pn_2241,pn_2242,pn_2243,pn_2244,pn_2245,pn_2246,pn_2247,pn_2248,pn_2249,pn_2250
topic_0,4.644312e-07,0.000000e+00,2.754238e-03,2.972032e-10,1.953082e-07,6.843103e-09,1.794003e-04,3.655916e-04,2.006316e-07,4.321228e-04,...,7.334924e-06,1.376734e-13,4.175594e-09,2.516665e-10,2.245302e-13,2.690170e-02,1.610970e-09,1.819347e-05,4.547948e-06,4.102871e-01
topic_1,4.700466e-09,0.000000e+00,1.109130e-08,3.390583e-10,1.852989e-07,1.231007e-07,1.775003e-08,3.833265e-11,1.172539e-07,1.852867e-08,...,7.536036e-10,2.851220e-09,5.944436e-05,6.284788e-10,4.007919e-05,1.438884e-05,9.995525e-01,1.172287e-08,4.968290e-09,1.087711e-07
topic_2,7.431173e-10,8.814978e-07,3.800897e-08,1.286532e-03,9.634998e-07,9.890727e-02,3.267656e-01,1.557679e-01,9.999953e-01,1.331347e-02,...,8.718837e-07,2.028473e-13,2.037030e-07,2.191602e-09,3.014792e-13,8.867306e-13,2.180929e-11,2.326771e-09,4.384303e-08,7.024916e-08
topic_3,9.995667e-01,0.000000e+00,2.203315e-01,8.552475e-01,1.242005e-06,3.948378e-08,8.331295e-05,3.710823e-12,8.086384e-07,4.442593e-01,...,9.769917e-02,4.605857e-13,4.695275e-09,6.217732e-10,2.534032e-12,2.197520e-01,6.136856e-11,1.047241e-01,1.749222e-01,5.472137e-01
topic_4,1.711281e-07,1.433753e-02,1.806813e-04,4.645708e-10,1.766514e-07,1.835193e-08,2.145215e-07,8.408949e-03,4.260514e-07,4.963123e-01,...,2.976635e-01,1.027876e-13,7.018269e-09,4.903777e-10,4.802517e-13,2.989310e-02,2.480177e-10,4.835853e-01,3.741977e-01,4.232885e-02
topic_5,4.488828e-10,0.000000e+00,1.076143e-08,1.415197e-09,9.996020e-01,6.470012e-08,9.989217e-09,4.124046e-09,3.329747e-07,9.675389e-09,...,6.019586e-11,4.146710e-13,2.380347e-09,8.668839e-01,1.503290e-13,8.346584e-10,6.349144e-08,4.767776e-07,3.037090e-09,4.112629e-08
topic_6,4.323343e-04,0.000000e+00,7.765557e-01,1.416761e-10,8.457881e-07,5.212289e-08,5.384067e-08,4.068769e-11,1.016402e-07,4.566724e-02,...,7.150330e-10,7.222894e-12,2.874997e-08,1.493271e-09,1.755746e-12,8.004991e-03,1.391589e-09,4.452746e-05,8.066996e-07,3.193658e-07
topic_7,7.562545e-10,0.000000e+00,1.832009e-07,3.131498e-10,2.010324e-07,1.017674e-07,3.307976e-05,1.500184e-11,1.255810e-07,2.564413e-08,...,1.338591e-08,4.815757e-13,4.524506e-08,2.023672e-09,8.177182e-01,7.949665e-11,3.021361e-10,7.045398e-09,5.670316e-08,1.841374e-07
topic_8,5.817632e-09,9.856616e-01,2.509079e-06,1.363564e-10,2.684362e-07,2.474734e-07,2.147702e-07,8.354574e-01,7.820233e-07,8.860453e-08,...,5.962839e-05,2.390424e-13,9.293336e-08,5.781985e-10,1.787278e-01,3.011034e-08,4.804061e-11,1.682097e-08,1.266494e-07,1.904067e-05
topic_9,2.589591e-09,0.000000e+00,6.590684e-09,9.938670e-10,5.557876e-07,7.610128e-02,1.476039e-07,1.290756e-11,1.228730e-07,5.073159e-08,...,8.196043e-10,8.344604e-12,4.140054e-07,1.208174e-09,1.424764e-03,7.763164e-02,2.888916e-10,3.244126e-09,4.997809e-08,4.608252e-08


In [91]:
level0.get_phi(class_ids=['text', 'flat_tag'])

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,background_0
пилотаж,0.000000e+00,1.766381e-13,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.343958e-05,0.000000e+00,9.704424e-08,0.000000e+00,0.000000e+00,0.000023
книголюб,0.000000e+00,1.704210e-13,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.708108e-05,0.000000e+00,3.392273e-08,0.000000e+00,0.000000e+00,0.000023
березайка,0.000000e+00,7.601469e-13,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.146338e-05,0.000000e+00,3.075504e-06,0.000000e+00,0.000000e+00,0.000023
паннонский,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.620454e-05,6.666219e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000023
šorъ,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.579176e-05,6.356475e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000023
латерал,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.624917e-05,5.147577e-09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000023
дендизм,0.000000e+00,0.000000e+00,0.000000e+00,1.909393e-12,0.000000e+00,0.000000e+00,0.000000e+00,1.188417e-03,2.506159e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
table,0.000000e+00,0.000000e+00,0.000000e+00,6.679670e-13,0.000000e+00,0.000000e+00,0.000000e+00,3.361865e-05,8.686628e-10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000023
дендистский,0.000000e+00,0.000000e+00,0.000000e+00,9.968050e-14,0.000000e+00,0.000000e+00,0.000000e+00,4.482555e-05,4.987781e-10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000023
sprezzatura,0.000000e+00,0.000000e+00,0.000000e+00,1.586958e-14,0.000000e+00,0.000000e+00,0.000000e+00,3.361887e-05,6.597772e-10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000023


### Стоп-слова

In [ ]:
tokens_list = []
unique_tokens = []
not_unique = []
stopwords = ''
for topic_name in level0.topic_names:
    unique_tokens = list(set(unique_tokens+tokens[topic_name]))
    tokens_list += tokens[topic_name]
    
counter = {word: 0 for word in unique_tokens}
for word in unique_tokens:
    for topic_name in level0.topic_names:
        if word in tokens[topic_name]:
            counter[word] +=1

for (word, n) in counter.items():
    if (n >= 3):
        stopwords += word + ' '
print(stopwords)

f = open('stopwords_0.txt', 'w')
f.write(stopwords)
f.close()

---

In [92]:
hier._levels.append(level0)

In [93]:
hier._levels

[artm.ARTM(num_topics=20, num_tokens=45093, class_ids=['flat_tag', 'text', 'author'])]

### Тут строится 1 уровень

In [94]:
hier.del_level(1)

In [95]:
norm_topic_names=['topic_'+str(i) for i in range(0,19*3)]
background_names=['background_'+str(i) for i in range(0,20)]
topic_names1 = norm_topic_names + background_names

In [96]:
topic_names1=['topic_'+str(i) for i in range(0,19*4+1)]

In [97]:
regularizers_list = []

#regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiAuthorReg',class_ids=['author'],tau=0.05))
#regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTagReg',class_ids=['flat_tag'],tau=-1))
#regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiReg',class_ids=['text'],tau=500000000))
regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiReg1',
                                                #topic_names=norm_topic_names,
                                                         class_ids=['text'],
                                                         tau=5000000))

#regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiRegB',
#                                                topic_names=background_names,
#                                                         class_ids=['text'],
#                                                         tau=5000000))

regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTextRegB',
                                                topic_names=topic_names1[-1:],
                                                         class_ids=['text'],
                                                         tau=1000))

#regularizers_list.append(artm.SmoothSparseThetaRegularizer(name='SmoothThetaRegSubj',
                                                           #topic_names=topics_subj, tau=100))

In [98]:
%%time

topicNum1 = (19*4)

level1 = hierarchy_utils.ARTM_Level(level0, phi_batch_weight=10.0**3, 
                                        num_topics=topicNum1,
                                        topic_names=topic_names1,
                                         class_ids=class_ids,
                                          regularizers=regularizers_list,
                                           scores=scores_list,
                                            num_document_passes=1, 
                                             cache_theta=True, seed=124)

CPU times: user 6.38 s, sys: 0 ns, total: 6.38 s
Wall time: 5.98 s


In [99]:
level1.regularizers

[DecorrPhiReg1, SPPhiTextRegB]

### Первые 5 итераций

In [100]:
%%time

level1.initialize(dictionary = dictionary)
level1.fit_offline(batch_vectorizer, num_collection_passes = 5)

CPU times: user 38.1 s, sys: 0 ns, total: 38.1 s
Wall time: 19.1 s


### Изменение регуляризаторов 

In [49]:
for i in range(len(level0.topic_names)):
    level1.regularizers.add(artm.DecorrelatorPhiRegularizer(name='SPPhiReg_from5it_'+str(i),
                                                topic_names=child_topics[i], 
                                                         tau=200000*len(child_topics[i])), overwrite=True) 
                    
    #level1.regularizers.add(artm.SmoothSparsePhiRegularizer(name='SPPhiReg_from5it_'+str(i), 
    #                                            topic_names=child_topics[i],
    #                                                     tau=-5*len(child_topics[i])**3), overwrite=True) 
    
    
#for k, name in enumerate(level1.topic_names):
#     if (name==child_topics[i][-1]):
#         level1.topic_names[k]='background_'+str(i)

NameError: name 'child_topics' is not defined

### Следующие 10 итераций

In [101]:
%%time

level1.fit_offline(batch_vectorizer, num_collection_passes = 10)

CPU times: user 1min 16s, sys: 0 ns, total: 1min 16s
Wall time: 37.7 s


### Вывод

In [102]:
tokens = level1.score_tracker["TopTokensTag"].last_tokens
output = ''
for topic_name in level1.topic_names:
    output += topic_name + ': '
    for word in tokens[topic_name]:    
        output += word + ' '
    output += '\n'
print(output)

topic_0: биология вирус мутация белки ген старение клеточная_биология нейродегенеративное_заболевание рнк кровеносная_система 
topic_1: ген биология белки старение рнк стволовые_клетки кардиология клеточная_биология фармакология крик_френсис 
topic_2: язык русский_язык римское_право английский_язык структура_языка невербальная_семиотика лексика вещное_право история_права городское_пространство 
topic_3: физика квантовая_физика квантовая_механика квантовые_технологии фотоны нанотехнологии магнитное_поле свет ньютон_исаак общая_теория_относительности 
topic_4: наука образование история_науки маркс_карл академическая_среда философия_науки социология_права феминизм социология_науки энергетика 
topic_5: язык русский_язык праязык римское_право фонетика английский_язык история_права полевая_лингвистика городское_пространство лингвистическая_компаративистика 
topic_6: технологии психология магнитное_поле поведение_человека личность когнитивная_психология внимание интеллект искусственный_интелл

In [34]:
print(level1.score_tracker['PerplexityScore'].value[-1])
print(level1.score_tracker['PerplexityScore_Tag'].value[-1])

867.8556783607152
30.89939739397064


### Стоп-слова 1

In [36]:
tokens_list = []
unique_tokens = []
not_unique = []
stopwords = ''
for topic_name in level1.topic_names:
    unique_tokens = list(set(unique_tokens+tokens[topic_name]))
    tokens_list += tokens[topic_name]
    
counter = {word: 0 for word in unique_tokens}
for word in unique_tokens:
    for topic_name in level1.topic_names:
        if word in tokens[topic_name]:
            counter[word] +=1

for (word, n) in counter.items():
    if (n >= 10):
        stopwords += word + ' '
print(stopwords)

f = open('stopwords_1.txt', 'w')
f.write(stopwords)
f.close()

история россия биология 


### Подтемы

In [104]:
psi1 = level1.get_psi()

In [105]:
threshold = 0.05
psi1  # p(subtopic|topic)

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,background_0
topic_0,0.000015,0.030216,0.000042,0.000110,0.000014,0.000064,0.000041,0.000068,0.000021,0.000042,0.075436,0.000174,0.000114,0.000261,0.000284,0.000298,0.001091,0.052732,0.015146,0.030229
topic_1,0.000599,0.003857,0.001925,0.001045,0.000067,0.000130,0.000041,0.000354,0.000210,0.000099,0.175622,0.000230,0.000997,0.007028,0.000705,0.000368,0.007400,0.006432,0.002062,0.004769
topic_2,0.000140,0.000249,0.000739,0.001022,0.000384,0.000025,0.237936,0.001553,0.000605,0.000793,0.000053,0.000150,0.000825,0.000079,0.000328,0.000082,0.003779,0.000408,0.000342,0.004259
topic_3,0.000375,0.000074,0.000857,0.000448,0.000064,0.001071,0.000191,0.000111,0.000379,0.004320,0.000429,0.132679,0.000341,0.005360,0.000107,0.008531,0.000153,0.000316,0.000185,0.000013
topic_4,0.000460,0.000298,0.001329,0.000514,0.000356,0.000595,0.001649,0.006631,0.001583,0.000255,0.001329,0.000426,0.252787,0.000239,0.010119,0.000280,0.000466,0.001020,0.000062,0.000016
topic_5,0.000464,0.000593,0.001425,0.001791,0.000805,0.000061,0.232327,0.003111,0.000818,0.000670,0.000095,0.000214,0.002388,0.000087,0.000396,0.001016,0.000910,0.001546,0.000053,0.000756
topic_6,0.000109,0.000936,0.000503,0.001474,0.000131,0.000170,0.001196,0.000182,0.000248,0.145684,0.000162,0.024931,0.000727,0.001688,0.000843,0.004955,0.000214,0.026886,0.005992,0.000338
topic_7,0.008543,0.000115,0.000941,0.000684,0.122798,0.000111,0.000378,0.000379,0.007564,0.000064,0.000046,0.000037,0.000848,0.000063,0.000679,0.000049,0.002613,0.000114,0.000196,0.000019
topic_8,0.000200,0.000141,0.000960,0.001752,0.000218,0.004233,0.000265,0.000217,0.000217,0.000618,0.001519,0.123854,0.001009,0.037005,0.000252,0.008217,0.000709,0.000325,0.000472,0.000018
topic_9,0.000144,0.000149,0.000443,0.001656,0.000165,0.000048,0.002175,0.156320,0.000382,0.000371,0.000632,0.000043,0.006912,0.000049,0.000274,0.000169,0.000046,0.000641,0.000237,0.003662


In [106]:
tokens0 = level0.score_tracker["TopTokensTag"].last_tokens
tokens1 = level1.score_tracker["TopTokensTag"].last_tokens

In [107]:
child_topics = []
output = ''
related = 0
for t, topic_name in enumerate(level0.topic_names):
    child_topics.append([])
    output += topic_name + ': '
    for word in tokens0[topic_name]:    
        output += word + ' '
    print("\b", output)
    output=''
    for s, topic_name1 in enumerate(level1.topic_names):
        if (psi1.values[s, t] > threshold):
        #&(int(topic_name1[-1])!=3):
            child_topics[t].append(topic_name1)
            related += 1
            output += "    "+ topic_name1 + ': '
            for word in tokens1[topic_name1]:    
                output += word + ' '
            print (output)
            output =''
print("\n edges: ", related)

related_topics = 0
for row in (psi1.values>threshold):
    if (sum(row) != 0):
        related_topics += 1

print("\n child topics: ", related_topics)    

 topic_0: россия история история_россии философия политическая_философия география античность история_философии аристотель археология 
    topic_14: россия география философия история_философии августин_аврелий бракосочетание история_россии аристотель платон политическая_философия 
    topic_23: история_россии россия история политическая_философия география платон история_философии русь вулканизм аристотель 
    topic_25: россия история_россии политическая_философия аристотель петр_i платон русская_философия политическая_теория римская_империя екатерина_ii 
    topic_52: история история_россии история_повседневности россия русь политический_режим кавказ просвещение география история_философии 
    topic_57: россия история_россии политическая_философия платон русь аристотель петр_i география история_философии русская_философия 
    topic_69: россия философия география политическая_философия древняя_греция история_философии платон история_россии аристотель екатерина_ii 
    topic_72: ге

In [108]:
child_topics

[['topic_14',
  'topic_23',
  'topic_25',
  'topic_52',
  'topic_57',
  'topic_69',
  'topic_72'],
 ['topic_16', 'topic_63', 'topic_64', 'topic_70'],
 ['topic_56'],
 ['topic_45', 'topic_61'],
 ['topic_7',
  'topic_17',
  'topic_22',
  'topic_34',
  'topic_41',
  'topic_49',
  'topic_51',
  'topic_52',
  'topic_58'],
 ['topic_29', 'topic_35', 'topic_50', 'topic_65'],
 ['topic_2', 'topic_5', 'topic_36', 'topic_40'],
 ['topic_9', 'topic_26', 'topic_48', 'topic_54', 'topic_62', 'topic_71'],
 ['topic_19', 'topic_30', 'topic_31', 'topic_60'],
 ['topic_6', 'topic_24', 'topic_33', 'topic_42', 'topic_43'],
 ['topic_0',
  'topic_1',
  'topic_12',
  'topic_20',
  'topic_39',
  'topic_44',
  'topic_75'],
 ['topic_3',
  'topic_8',
  'topic_10',
  'topic_11',
  'topic_15',
  'topic_33',
  'topic_46',
  'topic_74'],
 ['topic_4', 'topic_38', 'topic_53', 'topic_61', 'topic_68'],
 ['topic_47'],
 ['topic_37', 'topic_55', 'topic_61', 'topic_68'],
 ['topic_28', 'topic_66', 'topic_73', 'topic_74'],
 ['topic

In [109]:
tokens0 = level0.score_tracker["TopTokensScoreText"].last_tokens
tokens1 = level1.score_tracker["TopTokensScoreText"].last_tokens

In [110]:
output = ''
related = 0
for t, topic_name in enumerate(level0.topic_names):
    output += topic_name + ': '
    for word in tokens0[topic_name]:    
        output += word + ' '
    print("\b", output)
    output=''
    for s, topic_name1 in enumerate(level1.topic_names):
        if psi1.values[s, t] > threshold:
            related += 1
            output += "    "+ topic_name1 + ': '
            for word in tokens1[topic_name1]:    
                output += word + ' '
            print (output)
            output =''
print("\n edge: ", related)

 topic_0: россия русский политический философия российский государство война территория земля власть остров понятие место философ находить 
    topic_14: страна возникать спорт экономический право развитие хороший оставаться российский давать сторона идти место отношение правило 
    topic_23: археология украина землетрясение этика тирания рим обряд западничество океанология старообрядчество сибирь эстетика политический сокотра ксенофонт 
    topic_25: античность этика сократ архитектура археология рим риторика эстетика океан сокотра смерть землетрясение украина океанология ксенофонт 
    topic_52: власть питание начало политический смерть великий эпоха находить имя причина народ бог достаточно оставаться конец 
    topic_57: античность археология украина рим сократ тирания риторика ксенофонт смерть крепостничество язычество полис олимпиада землетрясение ленин 
    topic_69: рим филология русский риторика археология этика смерть обряд понятие украина классический мысль остров сибирь в

### Построение третьего уровня

In [279]:
hier.del_level(2)

In [280]:
hier._levels

[artm.ARTM(num_topics=20, num_tokens=45093, class_ids=['author', 'text', 'flat_tag'])]

In [560]:
regularizers_list = []

#regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiAuthorReg',class_ids=['author'],tau=0.05))
#regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTagReg',class_ids=['flat_tag'],tau=-1))
#regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiReg',class_ids=['text'],tau=500000000))
regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiReg1',
                                                         topic_names=[
                                                                      'topic_11','topic_12', 'topic_13', 'topic_14', 'topic_15',
                                                                      'topic_16', 'topic_17', 'topic_18', 'topic_19', 'topic_20',
                                                                     'topic_21', 'topic_22', 'topic_23', 'topic_24', 'topic_25',
                                                                      'topic_26', 'topic_27', 'topic_28', 'topic_29',
                                                                     'topic_30', 'topic_31', 'topic_32', 'topic_33', 'topic_34',
                                                                      'topic_35','topic_36', 'topic_37', 'topic_38', 'topic_39',
                                                                      'topic_40', 'topic_41', 'topic_42', 'topic_43', 'topic_44',
                                                                     'topic_45', 'topic_46', 'topic_47', 'topic_48', 'topic_49'],
                                                         tau=50000000))

regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiReg2',
                                                         topic_names=['topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5',
                                                                     'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10',],
                                                         tau=50000))

regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTextReg',
                                                         class_ids=['text'],
                                                         topic_names=['topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5',
                                                                     'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10',],
                                                         tau=5000))

#regularizers_list.append(artm.SmoothSparseThetaRegularizer(name='SmoothThetaRegSubj',
                                                           #topic_names=topics_subj, tau=100))

In [ ]:
%%time

topicNum2 = 50

level2 = hierarchy_utils.ARTM_Level(level1, phi_batch_weight=10.0**3, 
                                        num_topics=topicNum2,  
                                         class_ids=class_ids,
                                          regularizers=regularizers_list,
                                           scores=scores_list,
                                            num_document_passes=1, 
                                             cache_theta=True, seed=124)

In [ ]:
%%time

level2.initialize(dictionary=dictionary)
level2.fit_offline(batch_vectorizer, num_collection_passes=30)

In [ ]:
psi2 = level2.get_psi()
psi2>0.05

In [ ]:
output = ""
tokens2 = level2.score_tracker["TopTokensTag"].last_tokens
for topic_name in level2.topic_names:
    output += topic_name + ': '
    for word in tokens2[topic_name]:    
        output += word + " "
    output += "\n"
print(output)

In [ ]:
tokens0 = level1.score_tracker["TopTokensTag"].last_tokens
tokens1 = level2.score_tracker["TopTokensTag"].last_tokens

In [ ]:
threshold = 0.05

In [ ]:
output = ''
related = 0
for t, topic_name in enumerate(level1.topic_names):
    output += topic_name + ': '
    for word in tokens0[topic_name]:    
        output += word + ' '
    print("\b", output)
    output=''
    for s, topic_name1 in enumerate(level2.topic_names):
        if psi2.values[s, t] > threshold:
            related += 1
            output += "    "+ topic_name1 + ': '
            for word in tokens1[topic_name1]:    
                output += word + ' '
            print (output)
            output =''
print("\n edges: ", related)

### Сохронение (old)

In [114]:
hier._levels.append(level1)

In [292]:
hier._levels

[artm.ARTM(num_topics=20, num_tokens=45093, class_ids=['author', 'text', 'flat_tag']),
 artm.ARTM(num_topics=77, num_tokens=45093, class_ids=['author', 'text', 'flat_tag'])]

In [239]:
import pickle

In [49]:
%%time

# OUR OWN VERY COOL SAVE

dump = {}

for level_idx, level in enumerate(hier._levels):
    if level_idx > 0:
        # Zeroth level hierarchy doesn't have get_psi()
        dump["psi_%d" % level_idx] = level.get_psi()
    dump["phi_%d" % level_idx] = level.get_phi(class_ids='flat_tag')
dump["theta"] = hier.get_theta()

CPU times: user 1.48 s, sys: 0 ns, total: 1.48 s
Wall time: 1.37 s


In [579]:
dump = pickle.load(open("../server/hartm.mdl", "rb"))

In [580]:
level_1_columns = list(filter(lambda t: t.startswith("level1"), dump["theta"].index))

In [581]:
theta1 = dump["theta"].loc[level_1_columns]

In [582]:
doc_thresholds = theta1.max(axis=0) / np.sqrt(2)

In [612]:
topic_docs_count = theta1.apply(lambda s: sum(s >= doc_thresholds), axis=1)

In [617]:
topic_docs_count.get("level1_topic_0")

88

In [600]:
topic_id = "level1_topic_1"
ptd = theta1.loc[topic_id]

{k: v for k, v in ptd[ptd >= doc_thresholds].sort_values()[-10:][::-1].items()}

{'habr_110361': 0.12243211,
 'habr_83608': 0.16280058,
 'habr_87053': 0.1275439,
 'pn_1656': 0.11316308,
 'pn_2141': 0.14450958,
 'pn_2142': 0.15593959,
 'pn_244': 0.1529943,
 'pn_3331': 0.083700202,
 'pn_3392': 0.11312681,
 'pn_543': 0.097573817}

In [137]:
pickle.dump(dump, open("C:\\Users\\el1318\\Rysearch\\shared\\hartm.mdl", "wb"))

### Сохронение (new)

In [32]:
import json
import pickle
import urllib.request

In [127]:
# Phi, Psi

hier.save("hartm")

In [129]:
# Extra info

extra_info = {
    "class_ids": class_ids,
    "theta": hier.get_theta(),
}

pickle.dump(extra_info, open("hartm/extra_info.dump", "wb"))

---

In [50]:
theta_old = hier.get_theta().T

NameError: name 'hier' is not defined

In [567]:
level_0_columns = list(filter(lambda t: t.startswith("level_0_topic_"), theta_old.columns))

In [568]:
theta0_old = theta_old[level_0_columns]

In [570]:
hier._levels

[artm.ARTM(num_topics=20, num_tokens=45093, class_ids=['author', 'text', 'flat_tag']),
 artm.ARTM(num_topics=77, num_tokens=45093, class_ids=['author', 'text', 'flat_tag'])]

In [62]:
%%time

hier_new = hierarchy_utils.hARTM(theta_columns_naming="title", class_ids=class_ids)

hier_new.load("hartm")

CPU times: user 7.15 s, sys: 0 ns, total: 7.15 s
Wall time: 6.91 s


In [49]:
hierarchy_utils

<module 'hierarchy_utils' from '/home/anton/Projects/Rysearch/experiments/hierarchy_utils.py'>

In [11]:
hier_new._levels

[artm.ARTM(num_topics=20, num_tokens=45093, class_ids=['flat_tag', 'text']),
 artm.ARTM(num_topics=77, num_tokens=45093, class_ids=['flat_tag', 'text'])]

In [130]:
%%time

test_batch_vectorizer = artm.BatchVectorizer(data_format="vowpal_wabbit",
                                             data_path = "test.txt",
                                             batch_size = 225,
                                             target_folder="test_batches/",)

CPU times: user 7.28 s, sys: 0 ns, total: 7.28 s
Wall time: 2.42 s


In [131]:
%%time

test_df = hier_new.transform(test_batch_vectorizer).T

CPU times: user 10.6 s, sys: 0 ns, total: 10.6 s
Wall time: 3.02 s


In [134]:
test_df.T

,pn_2926,pn_2927,pn_2928,pn_2929,pn_2930,pn_2931,pn_2932,pn_2933,pn_2934,pn_2935,...,pn_441,pn_442,pn_443,pn_444,pn_445,pn_446,pn_447,pn_448,pn_449,pn_450
level_0_topic_0,2.208498e-12,1.541284e-14,1.038942e-12,2.754957e-12,2.976427e-13,1.450592e-05,2.321602e-08,1.598011e-02,7.640131e-05,4.417442e-03,...,3.196012e-04,0.000000e+00,4.778791e-15,0.000000e+00,6.017108e-04,1.923412e-15,0.000000,2.022263e-14,2.730265e-03,0.000000e+00
level_0_topic_1,3.282694e-12,2.446118e-09,3.667514e-11,2.373219e-05,1.668725e-11,1.530821e-03,3.838088e-08,4.700103e-07,1.022535e-08,1.709126e-03,...,0.000000e+00,0.000000e+00,3.190175e-15,0.000000e+00,2.569073e-16,6.913443e-15,0.000000,3.385637e-14,2.954398e-16,1.188740e-11
level_0_topic_2,2.454799e-13,8.905348e-14,1.497724e-10,2.082680e-10,1.270770e-12,1.865636e-04,3.878114e-09,1.857902e-05,7.845704e-07,2.770143e-03,...,0.000000e+00,0.000000e+00,2.187219e-12,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,9.936289e-15,2.242526e-04,0.000000e+00
level_0_topic_3,3.322270e-12,9.401876e-14,1.430700e-11,5.438373e-10,1.573938e-10,4.141862e-05,1.486228e-09,9.762986e-01,2.186698e-01,3.868013e-04,...,4.572782e-06,7.006971e-15,2.338817e-13,0.000000e+00,8.555925e-06,3.917397e-14,0.000000,1.213578e-13,1.323429e-03,0.000000e+00
level_0_topic_4,5.130014e-13,1.888034e-14,2.338648e-12,1.524310e-11,6.502362e-13,3.460311e-06,9.590354e-02,2.461978e-05,1.762934e-08,9.645806e-04,...,0.000000e+00,0.000000e+00,5.163956e-14,0.000000e+00,0.000000e+00,1.765322e-15,0.000000,2.449103e-14,1.963434e-01,0.000000e+00
level_0_topic_5,1.008109e-11,2.880016e-13,9.318194e-12,3.574961e-11,1.798060e-14,1.547644e-05,9.228158e-10,6.199097e-08,6.357788e-10,6.289978e-01,...,0.000000e+00,0.000000e+00,1.881948e-15,0.000000e+00,0.000000e+00,1.493116e-15,0.000000,2.608799e-14,1.764347e-06,0.000000e+00
level_0_topic_6,1.733148e-10,7.693032e-14,3.506916e-11,1.488700e-08,8.742022e-13,5.565864e-03,5.304263e-09,6.960635e-03,1.964541e-01,5.331229e-04,...,1.509176e-13,0.000000e+00,6.610287e-10,0.000000e+00,1.354976e-16,1.223017e-14,0.000000,7.746390e-14,1.995540e-13,0.000000e+00
level_0_topic_7,1.378861e-11,3.506718e-14,1.456349e-11,7.946677e-11,1.364848e-06,4.429001e-04,1.102413e-01,3.333990e-06,4.345160e-01,1.441712e-04,...,9.663708e-01,1.000000e+00,9.999999e-01,9.998550e-01,9.017791e-01,8.065342e-01,1.000000,1.000000e+00,3.974523e-11,5.400618e-07
level_0_topic_8,1.592312e-12,1.036130e-14,9.656938e-12,1.142452e-10,1.260087e-02,1.755580e-04,1.497209e-01,8.495766e-05,1.502768e-01,2.664991e-03,...,0.000000e+00,0.000000e+00,2.449570e-14,1.449667e-04,6.732640e-05,2.444672e-15,0.000000,4.880080e-14,9.406745e-06,0.000000e+00
level_0_topic_9,2.452180e-01,4.166658e-14,1.505763e-02,2.612768e-10,1.688012e-01,5.762813e-03,2.272705e-01,3.132156e-06,5.576057e-09,2.520792e-03,...,0.000000e+00,1.013889e-16,6.476045e-14,0.000000e+00,2.300988e-16,2.180983e-14,0.000000,1.350610e-13,4.046284e-01,0.000000e+00


In [21]:
level_0_columns = list(filter(lambda t: t.startswith("level_0_topic"), test_df.columns))

In [22]:
theta0 = test_df[level_0_columns]

In [23]:
theta0.shape

(3446, 19)

In [25]:
# Получим информацию о темах

request_url = "http://localhost:3000/get-topics"
response = urllib.request.urlopen(request_url).read().decode("utf-8")
topics_data = json.loads(response)

In [28]:
bad_ids = set()

for i, (doc_id, doc_topics) in enumerate(test_df[level_0_columns].iterrows()):
    # Получим информацию о документе
    request_url = "http://localhost:3000/get-document?doc_id=%s" % doc_id
    response = urllib.request.urlopen(request_url).read().decode("utf-8")
    doc_info = json.loads(response)
    #print("%s: %s" % (doc_id, doc_info["title"]))

    topics = doc_topics.sort_values()[-3:][::-1]
    for topic_id, topic_weight in topics.items():
        topic_name = ", ".join(topics_data[topic_id]["top_words"])
        topic_weight_old = theta0_old[doc_id][topic_id]
        if np.abs(topic_weight - topic_weight_old) > 0.1:
            pass #print("(%s) %s: %s (%.1f%% / %.1f%%)" % (doc_id, topic_id, topic_name, 100 * topic_weight, 100 * topic_weight_old))
            bad_ids.add(doc_id)
    
    #if i == 100: break
    
print(len(bad_ids))

NameError: name 'theta0_old' is not defined

### Grid search

In [ ]:
from sklearn.model_selection import ParameterGrid

In [ ]:
param_grid = {'SPPhiAuthorReg': [0],'SPPhiTagReg': [-5, -2, -0.5, -0.05, 0, 0.05, 0.5, 2, 5], 
             'DecorrPhiReg': [500000000], 'SPPhiTextReg': [-5, -2, -0.5, -0.05, 0, 0.05, 0.5, 2, 5],
             'SmoothThetaReg': [-10, -5, -2, -1, 0, 1, 2, 5, 10] }

In [ ]:
parameters = list(ParameterGrid(param_grid))

In [ ]:
hier.regularizers

In [ ]:
regularizers_names = [regularizers_list[i].name for i in range(0, len(regularizers_list))]
regularizers_list[0]._tau = 1
regularizers_list[0].tau

In [ ]:
%%time

params_textprepl = {}
params_tagsprepl = {}

for taus in parameters:
    for name, value in taus.items():
        for i in range(0, len(regularizers_list)):
            if(regularizers_list[i].name == name):
                regularizers_list[i]._tau = value
    level0.fit_offline(batch_vectorizer, num_collection_passes=10)
    params_textprepl[str(taus)]=level0.score_tracker['PerplexityScore'].value[-1]
    params_tagsprepl[str(taus)]=level0.score_tracker['PerplexityScore_Tag'].value[-1]

In [ ]:
import operator
#print(sorted(params_tagsprepl))
print(sorted(params_textprepl.items(), key=operator.itemgetter(1)))

In [ ]:
print(len(params_textprepl))

In [ ]:
param_grid = {'SPPhiAuthorReg': [0],'SPPhiTagReg': [-0.2, -0.15, -0.1, -0.05, 0, 0.05, 0.1, 0.15, 0.2], 
             'DecorrPhiReg': [500000000], 'SPPhiTextReg': [-0.2, -0.15, -0.1, -0.05, 0, 0.05, 0.1, 0.15, 0.2],
             'SmoothThetaReg': [-0.2, -0.15, -0.1, -0.05, 0, 0.05, 0.1, 0.15, 0.2] }

In [ ]:
%%time

params_textprepl1 = {}
params_tagsprepl1 = {}

for taus in parameters:
    for name, value in taus.items():
        for i in range(0, len(regularizers_list)):
            if(regularizers_list[i].name == name):
                regularizers_list[i]._tau = value
    level0.fit_offline(batch_vectorizer, num_collection_passes=10)
    params_textprepl1[str(taus)]=level0.score_tracker['PerplexityScore'].value[-1]
    params_tagsprepl1[str(taus)]=level0.score_tracker['PerplexityScore_Tag'].value[-1]

In [ ]:
#print(sorted(params_tagsprepl))
print(sorted(params_textprepl.items(), key=operator.itemgetter(1)))

In [ ]:
print(len(params_textprepl1))

In [ ]:
print(sorted(params_tagsprepl1.items(), key=operator.itemgetter(1)))

### Модификация словаря

In [ ]:
#создает словарь, в котором для слов из tokens value=1, а для остальных токенов value=0
def create_my_dict(tokens):
    line='newline'
    with open('E:/lib/ARTM/htest/UCI_collections/UCI_trimmed/dictionary.txt', 'r', encoding='utf-8', errors='replace') as dict0, open('E:/lib/ARTM/htest/UCI_collections/UCI_trimmed/my_dict'+topic_name[-2:]+'.txt', 'w+') as my_dict:
        while (line!=''):
            try:
                line=dict0.readline()
                line=line.split(', ')
                if(len(line)==5):
                    if (line[0] in tokens):
                        line[1] = '1.0'
                    else:
                        line[1] = '0.0'
                line = ', '.join(line)
                my_dict.write(line)
            except UnicodeEncodeError:
                pass
            except UnicodeDecodeError:
                pass
    my_dict = artm.Dictionary('mydict'+topic_name[-2:])
    my_dict.load_text('E:/lib/ARTM/htest/UCI_collections/UCI_trimmed/my_dict'+topic_name[-2:]+'.txt', encoding = 'cp1251')
    print(my_dict)
    return my_dict
    